# Yelp API - Lab


## Introduction 

We've seen how the Yelp API works and how to create basic visualizations using Folium. It's time to put those skills to work in order to create a working map! Taking things a step further, you'll also independently explore how to perform pagination in order to retrieve a full results set from the Yelp API!

## Objectives

You will be able to: 

* Using pagination and multiple functions, gather large amounts of data from an API, parse the data and make sense of it with meaningful analysis
* Create maps using Folium

## Problem Introduction

You've now worked with some API calls, but we have yet to see how to retrieve a more complete dataset in a programmatic manner. Returning to the Yelp API, the [documentation](https://www.yelp.com/developers/documentation/v3/business_search) also provides us details regarding the API limits. These often include details about the number of requests a user is allowed to make within a specified time limit and the maximum number of results to be returned. In this case, we are told that any request has a maximum of 50 results per request and defaults to 20. Furthermore, any search will be limited to a total of 1000 results. To retrieve all 1000 of these results, we would have to page through the results piece by piece, retrieving 50 at a time. Processes such as these are often referred to as pagination.

In this lab, you will define a search and then paginate over the results to retrieve all of the results. You'll then parse these responses as a DataFrame (for further exploration) and create a map using Folium to visualize the results geographically.

## Part I - Make the Initial Request

Start by making an initial request to the Yelp API. Your search must include at least 2 parameters: **term** and **location**. For example, you might search for pizza restaurants in NYC. The term and location is up to you but make the request below.

In [1]:
import json

def get_keys(path):
    with open(path) as f:
        return json.load(f)

In [2]:
keys = get_keys("/Users/wvsharber/.secret/yelp_api.json")

api_key = keys['api_key']

In [3]:
import requests
term = 'pizza'
location = 'Tulsa OK'
SEARCH_LIMIT = 1

url = 'https://api.yelp.com/v3/businesses/search'

headers = {
        'Authorization': 'Bearer {}'.format(api_key),
    }

url_params = {
                'term': term.replace(' ', '+'),
                'location': location.replace(' ', '+'),
                'limit': SEARCH_LIMIT
            }
response = requests.get(url, headers=headers, params=url_params)

In [4]:
print(response)
print(type(response.text))
print(response.text[:1000])

<Response [200]>
<class 'str'>
{"businesses": [{"id": "jFBPuK0LwDdDYSOVP6EBGw", "alias": "andolinis-pizzeria-tulsa", "name": "Andolini's Pizzeria", "image_url": "https://s3-media2.fl.yelpcdn.com/bphoto/rZvJ0-PaGxynYyGr6fXN6w/o.jpg", "is_closed": false, "url": "https://www.yelp.com/biz/andolinis-pizzeria-tulsa?adjust_creative=GfrfRbyX2iYO3M0rvKOQeg&utm_campaign=yelp_api_v3&utm_medium=api_v3_business_search&utm_source=GfrfRbyX2iYO3M0rvKOQeg", "review_count": 657, "categories": [{"alias": "pizza", "title": "Pizza"}, {"alias": "bars", "title": "Bars"}, {"alias": "italian", "title": "Italian"}], "rating": 4.5, "coordinates": {"latitude": 36.14027, "longitude": -95.96984}, "transactions": ["delivery"], "price": "$$", "location": {"address1": "1552 E 15th St", "address2": "", "address3": "", "city": "Tulsa", "zip_code": "74120", "country": "US", "state": "OK", "display_address": ["1552 E 15th St", "Tulsa, OK 74120"]}, "phone": "+19187286111", "display_phone": "(918) 728-6111", "distance": 571

In [5]:
response.json()['total']

352

## Pagination

Now that you have an initial response, you can examine the contents of the JSON container. For example, you might start with ```response.json().keys()```. Here, you'll see a key for `'total'`, which tells you the full number of matching results given your query parameters. Write a loop (or ideally a function) which then makes successive API calls using the offset parameter to retrieve all of the results (or 5000 for a particularly large result set) for the original query. As you do this, be mindful of how you store the data. Your final goal will be to reformat the data concerning the businesses themselves into a pandas DataFrame from the json objects.

**Note: be mindful of the API rate limits. You can only make 5000 requests per day and are also can make requests too fast. Start prototyping small before running a loop that could be faulty. You can also use time.sleep(n) to add delays. For more details see https://www.yelp.com/developers/documentation/v3/rate_limiting.**

In [7]:
# Your code here; use a function or loop to retrieve all the results from your original request

#write for loop that gets 50 results at a time until 3400 (may need to limit how quickly each get request is sent)
#include error processing
#process the business part of the response into a pandas DataFrame
#concatenate each 50 results onto the DataFrame



In [25]:
import pandas as pd
import time

#need to add if statement to pause before retrieving >1000 results

def get_yelp_results(searchterm, searchlocation):
    """This function returns the search results in a Pandas DataFrame
       from Yelp api based on a search term and search location"""
    term = searchterm
    location = searchlocation
    SEARCH_LIMIT = 50
    offset = 0
    
    url = 'https://api.yelp.com/v3/businesses/search'

    headers = {
        'Authorization': 'Bearer {}'.format(api_key),
        }

    url_params = {
                'term': term.replace(' ', '+'),
                'location': location.replace(' ', '+'),
                'limit': SEARCH_LIMIT,
                'offset': offset
                }
    
    initialresponse = requests.get(url, headers=headers, params=url_params)
    businesses = pd.DataFrame.from_dict(initialresponse.json()['businesses'])
#   if initialresponse.json()['total'] <= 1000:
    while len(businesses) < initialresponse.json()['total'] and len(businesses) < 1000:
        offset += 50
        url_params = {
                'term': term.replace(' ', '+'),
                'location': location.replace(' ', '+'),
                'limit': SEARCH_LIMIT,
                'offset': offset
                }
        response = requests.get(url, headers=headers, params=url_params)
        if response.status_code == 200:
            bus50 = pd.DataFrame.from_dict(response.json()['businesses'])
            businesses = pd.concat([businesses, bus50], ignore_index = True)
            print(offset, "businesses returned")
            time.sleep(0.5)

        elif response.status_code == 400:
            print("Reached max number of businesses returned")
            break 
        
        else:
            print("Error: unable to retrieve businesses. Server responded with status code", response.status_code)
            break
#     else:
#         for i in range(0, initialresponse.json()['total'], 1000): 
#             while len(businesses) < initialresponse.json()['total'] and len(businesses) <= i * 1000:
#                     offset += 50
#                     url_params = {
#                             'term': term.replace(' ', '+'),
#                             'location': location.replace(' ', '+'),
#                             'limit': SEARCH_LIMIT,
#                             'offset': offset
#                             }
#                     response = requests.get(url, headers=headers, params=url_params)
#                     if response.status_code == 200:
#                         bus50 = pd.DataFrame.from_dict(response.json()['businesses'])
#                         businesses = pd.concat([businesses, bus50], ignore_index = True)
#                         print(offset)
#                         time.sleep(0.5)

#                     else:
#                         print("Error: unable to retrieve businesses. Server responded with status code", response.status_code)
#                         break
#             time.sleep(60)
#             print("Waiting 1 min to retrieve next 1000 results")
    return businesses
        

In [21]:
tulsapizza = get_yelp_results(searchterm="pizza", searchlocation="Tulsa OK")

50 number of businesses returned
100 number of businesses returned
150 number of businesses returned
200 number of businesses returned
250 number of businesses returned
300 number of businesses returned
350 number of businesses returned


In [35]:
seattlecoffee = get_yelp_results("coffee", "Seattle WA")

50 businesses returned
100 businesses returned
150 businesses returned
200 businesses returned
250 businesses returned
300 businesses returned
350 businesses returned
400 businesses returned
450 businesses returned
500 businesses returned
550 businesses returned
600 businesses returned
650 businesses returned
700 businesses returned
750 businesses returned
800 businesses returned
850 businesses returned
900 businesses returned
950 businesses returned


## Exploratory Analysis

Take the restaurants from the previous question and do an initial exploratory analysis. At minimum, this should include looking at the distribution of features such as price, rating and number of reviews as well as the relations between these dimensions.

In [37]:
seattlecoffee.head()

,alias,categories,coordinates,display_phone,distance,id,image_url,is_closed,location,name,phone,price,rating,review_count,transactions,url
0,half-and-half-doughnut-seattle,"[{'alias': 'coffee', 'title': 'Coffee & Tea'},...","{'latitude': 47.61425, 'longitude': -122.32478}",(206) 325-5509,1479.893518,I3R01dj_8HzFb57VBmt30Q,https://s3-media2.fl.yelpcdn.com/bphoto/DEnDz2...,False,"{'address1': '516 E PIke St', 'address2': None...",Half and Half Doughnut,+12063255509,NaN,4.5,27,[],https://www.yelp.com/biz/half-and-half-doughnu...
1,storyville-coffee-company-seattle-9,"[{'alias': 'coffee', 'title': 'Coffee & Tea'}]","{'latitude': 47.6088847, 'longitude': -122.340...",(206) 780-5777,1874.645904,FVzl8rDPiTWEtrNEuCu-Xg,https://s3-media3.fl.yelpcdn.com/bphoto/nJgiyj...,False,"{'address1': '94 Pike St', 'address2': 'Ste 34...",Storyville Coffee Company,+12067805777,$$,4.5,1586,"[pickup, delivery]",https://www.yelp.com/biz/storyville-coffee-com...
2,seattle-coffee-works-seattle,"[{'alias': 'coffeeroasteries', 'title': 'Coffe...","{'latitude': 47.6101133, 'longitude': -122.340...",(206) 340-8867,1743.611549,RwCpDB4C7CCFQvFxH1ve1g,https://s3-media1.fl.yelpcdn.com/bphoto/nDzPH0...,False,"{'address1': '108 Pine St', 'address2': None, ...",Seattle Coffee Works,+12063408867,$,4.5,856,[],https://www.yelp.com/biz/seattle-coffee-works-...
3,moore-coffee-shop-seattle,"[{'alias': 'coffee', 'title': 'Coffee & Tea'},...","{'latitude': 47.611705, 'longitude': -122.3412...",(206) 883-7044,1584.516419,qKkHnsG-f4-BhdkR6yRrgw,https://s3-media4.fl.yelpcdn.com/bphoto/HTHJUX...,False,"{'address1': '1930 2nd Ave', 'address2': None,...",Moore Coffee Shop,+12068837044,$,4.5,1041,[],https://www.yelp.com/biz/moore-coffee-shop-sea...
4,anchorhead-coffee-seattle,"[{'alias': 'coffee', 'title': 'Coffee & Tea'},...","{'latitude': 47.6133808022766, 'longitude': -1...",(206) 222-2222,1339.997321,K-X3rRAVbeMZ0VIDWuPX_Q,https://s3-media1.fl.yelpcdn.com/bphoto/bLlhUj...,False,"{'address1': '1600 7th Ave', 'address2': 'Ste ...",Anchorhead Coffee,+12062222222,$,4.5,452,[],https://www.yelp.com/biz/anchorhead-coffee-sea...


In [49]:
prices = seattlecoffee['price'].dropna()

In [50]:
prices = pd.Series([len(x) for x in prices])

In [52]:
print(prices.mean(), prices.median(), prices.mode())

1.4063604240282686 1.0 0    1
dtype: int64


In [36]:
for x in seattlecoffee['price']:
    if x.isna() == False:
        x = len(str(x))
    else:
        next

AttributeError: 'float' object has no attribute 'isna'

In [34]:
seattlecoffee.head()

,alias,categories,coordinates,display_phone,distance,id,image_url,is_closed,location,name,phone,price,rating,review_count,transactions,url
0,storyville-coffee-company-seattle-9,"[{'alias': 'coffee', 'title': 'Coffee & Tea'}]","{'latitude': 47.6088847, 'longitude': -122.340...",(206) 780-5777,1874.645904,FVzl8rDPiTWEtrNEuCu-Xg,https://s3-media3.fl.yelpcdn.com/bphoto/nJgiyj...,False,"{'address1': '94 Pike St', 'address2': 'Ste 34...",Storyville Coffee Company,+12067805777,2,4.5,1586,"[delivery, pickup]",https://www.yelp.com/biz/storyville-coffee-com...
1,half-and-half-doughnut-seattle,"[{'alias': 'coffee', 'title': 'Coffee & Tea'},...","{'latitude': 47.61425, 'longitude': -122.32478}",(206) 325-5509,1479.893518,I3R01dj_8HzFb57VBmt30Q,https://s3-media2.fl.yelpcdn.com/bphoto/DEnDz2...,False,"{'address1': '516 E PIke St', 'address2': None...",Half and Half Doughnut,+12063255509,3,4.5,27,[],https://www.yelp.com/biz/half-and-half-doughnu...
2,seattle-coffee-works-seattle,"[{'alias': 'coffeeroasteries', 'title': 'Coffe...","{'latitude': 47.6101133, 'longitude': -122.340...",(206) 340-8867,1743.611549,RwCpDB4C7CCFQvFxH1ve1g,https://s3-media1.fl.yelpcdn.com/bphoto/nDzPH0...,False,"{'address1': '108 Pine St', 'address2': None, ...",Seattle Coffee Works,+12063408867,1,4.5,856,[],https://www.yelp.com/biz/seattle-coffee-works-...
3,moore-coffee-shop-seattle,"[{'alias': 'coffee', 'title': 'Coffee & Tea'},...","{'latitude': 47.611705, 'longitude': -122.3412...",(206) 883-7044,1584.516419,qKkHnsG-f4-BhdkR6yRrgw,https://s3-media4.fl.yelpcdn.com/bphoto/HTHJUX...,False,"{'address1': '1930 2nd Ave', 'address2': None,...",Moore Coffee Shop,+12068837044,1,4.5,1041,[],https://www.yelp.com/biz/moore-coffee-shop-sea...
4,milstead-seattle-3,"[{'alias': 'coffee', 'title': 'Coffee & Tea'}]","{'latitude': 47.6493630890404, 'longitude': -1...",(206) 659-4814,2800.041516,H_lQTUTQm9K0tmEJLZ4LAQ,https://s3-media1.fl.yelpcdn.com/bphoto/34C4_y...,False,"{'address1': '754 N 34th St', 'address2': None...",Milstead,+12066594814,1,4.5,433,[],https://www.yelp.com/biz/milstead-seattle-3?ad...


## Mapping

Look at the initial Yelp example and try and make a map using Folium of the restaurants you retrieved. Be sure to also add popups to the markers giving some basic information such as name, rating and price.

In [ ]:
# Your code here

## Summary

Nice work! In this lab, you've made multiple API calls to Yelp in order to paginate through a results set, performing some basic exploratory analysis and then creating a nice interactive map to display the results using Folium! Well done!